In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow import keras 
from tensorflow.keras import layers
import os
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, BatchNormalization, AveragePooling2D, Flatten
from sklearn import svm 
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
import aspose.words as aw

In [2]:
"""Getting df with all filenames and images (missing labels at this stage) - shape = (166866,2)"""
# only take first ten photos in each brand folder
# (150,150) grayscale --> 2m 28.9s
# (150,150) rgb, with pixels as cols --> 

def load_images_from_folder(folder_path, image_size=(150, 150)):
    images = []
    
    # Recursively traverse through the folder structure
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            # Get the full path of the file
            file_path = os.path.join(root, filename)
            
            # Check if the file is an image
            if any(filename.endswith(extension) for extension in ['.jpg', '.jpeg', '.png', '.gif']):
                # Load the image using PIL (Python Imaging Library)
                try:
                    with Image.open(file_path) as img:
                        img = img.resize(image_size) #resizing image
                        img = img.convert('RGB') #Converting to gray scale
                        #img_array = np.array(img) / 255.0 # Convert the image to numpy array and normalize pixel values
                         # Extract folder name from the root path
                        label = os.path.basename(root)
                        
                        images.append({'Label': label, 'Filename': filename, 'Image': img})
                        #images.append({'Filename': filename, 'Image': img}) # Append the image array to the list
                except Exception as e:
                    print(f"Error loading image '{file_path}': {e}")
    
    # Convert the list of dictionaries to a DataFrame
    images_df = pd.DataFrame(images)
    return images_df

# Loading
png_folder_path=r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder"
image_df = load_images_from_folder(png_folder_path)
print('Image Dataframe: \n \n', image_df.head())

C:\Users\emili\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Error loading image 'C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder\Original Aston Martin logo\Image_30.png': Decompressed Data Too Large
Error loading image 'C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder\Original Maserati logo\Image_13.png': Decompressed Data Too Large
Error loading image 'C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder\Original Porsche logo\Image_18.png': Decompressed Data Too Large
Image Dataframe: 
 
                         Label      Filename  \
0  Original Aston Martin logo  Image_10.png   
1  Original Aston Martin logo  Image_11.png   
2  Original Aston Martin logo  Image_12.png   
3  Original Aston Martin logo  Image_13.png   
4  Original Aston Martin logo  Image_15.png   

                                               Image  
0  <PIL.Image.Image image mode=RGB size=150x150 a...  
1  <PIL.Image.Im

In [3]:
label_encoder = LabelEncoder()
image_df['Label coded'] = label_encoder.fit_transform(image_df['Label'])
image_df

,Label,Filename,Image,Label coded
0,Original Aston Martin logo,Image_10.png,<PIL.Image.Image image mode=RGB size=150x150 a...,0
1,Original Aston Martin logo,Image_11.png,<PIL.Image.Image image mode=RGB size=150x150 a...,0
2,Original Aston Martin logo,Image_12.png,<PIL.Image.Image image mode=RGB size=150x150 a...,0
3,Original Aston Martin logo,Image_13.png,<PIL.Image.Image image mode=RGB size=150x150 a...,0
4,Original Aston Martin logo,Image_15.png,<PIL.Image.Image image mode=RGB size=150x150 a...,0
...,...,...,...,...
364,Original Porsche logo,Image_5.png,<PIL.Image.Image image mode=RGB size=150x150 a...,13
365,Original Porsche logo,Image_6.png,<PIL.Image.Image image mode=RGB size=150x150 a...,13
366,Original Porsche logo,Image_7.png,<PIL.Image.Image image mode=RGB size=150x150 a...,13
367,Original Porsche logo,Image_8.png,<PIL.Image.Image image mode=RGB size=150x150 a...,13


In [4]:
#split it into features and labels
images = image_df.drop(['Filename','Label coded','Label'], axis=1)
labels = image_df[['Label coded']]

In [5]:
X_train ,X_test, y_train, y_test=train_test_split(images, labels, test_size=0.2, random_state=42)

In [6]:
# perform PCA on X to reduce amount of features
# reduce to dimensionality so that 80% of variance is kept
from sklearn.decomposition import PCA

# Convert PIL images/tensors in DataFrame to NumPy array
X_train = np.array([np.array(img) for img in X_train.iloc[:, 0]])
X_test = np.array([np.array(img) for img in X_test.iloc[:, 0]])

# reshape features accordingly to SVM architecture NOTE: train test split again!!
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

pca = PCA(n_components=0.8, random_state=42)
X_pca_train = pca.fit_transform(X_train)

# check how many principal components are needed
pca.n_components_

# apply the trained PCA model on the test set as well
X_pca_test = pca.transform(X_test)

# check that PCA worked
print(X_pca_train.shape)
print(X_pca_test.shape)

(295, 24)
(74, 24)


In [7]:
# define the linear SVM model
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import classification_report, multilabel_confusion_matrix


svm = SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42)
multi_svm = ClassifierChain(svm)

# define the hyperparameter grid
param_grid = {
    # the selection of the hyperparameters was in the beginning larger and reduced after some tries
    # restricting the model with smaller values made the best results
    'base_estimator__C':[2**-5, 2**-4, 2**-3, 2**-1],
}

# define the scoring metric
scorer = make_scorer(roc_auc_score, multi_class='ovr', average='macro')

# define the grid search
grid = GridSearchCV(
    multi_svm,
    param_grid,
    cv=5,
    scoring=scorer,
    refit=True,
    return_train_score=True,
    verbose=3)

In [8]:
# train the SVMs
grid.fit(X_pca_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda

[CV 1/5] END base_estimator__C=0.03125;, score=(train=nan, test=nan) total time=   0.1s


C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda

[CV 2/5] END base_estimator__C=0.03125;, score=(train=nan, test=nan) total time=   0.2s


...

C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda

[CV 4/5] END base_estimator__C=0.5;, score=(train=nan, test=nan) total time=   0.2s


C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emili\anaconda

[CV 5/5] END base_estimator__C=0.5;, score=(train=nan, test=nan) total time=   0.1s


GridSearchCV(cv=5,
             estimator=ClassifierChain(base_estimator=SVC(class_weight='balanced',
                                                          kernel='linear',
                                                          probability=True,
                                                          random_state=42)),
             param_grid={'base_estimator__C': [0.03125, 0.0625, 0.125, 0.5]},
             return_train_score=True,
             scoring=make_scorer(roc_auc_score, multi_class=ovr, average=macro),
             verbose=3)

In [9]:
# check performance of the models
pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_train_score"], columns=["Training Mean AUC"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["Validation Mean AUC"])], axis=1)

,base_estimator__C,Training Mean AUC,Validation Mean AUC
0,0.03125,NaN,NaN
1,0.06250,NaN,NaN
2,0.12500,NaN,NaN
3,0.50000,NaN,NaN


In [10]:
# print the best hyperparameters
print('Best Hyperparameters: ', grid.best_params_)

Best Hyperparameters:  {'base_estimator__C': 0.03125}


In [12]:
# make predictions on the test set (probabilities)
y_pred_prob = grid.predict_proba(X_pca_test)

In [13]:
# get auc from the test set
auc_score = roc_auc_score(y_test, y_pred_prob, average='macro')

ValueError: multi_class must be in ('ovo', 'ovr')

In [14]:
# check predictions (probabilities)
y_pred_prob[:5].round(2)

array([[0.07],
       [0.1 ],
       [0.07],
       [0.05],
       [0.07]])

In [15]:
# make predictions on the test set
y_pred = grid.predict(X_pca_test)

ValueError: X has 67500 features, but ClassifierChain is expecting 24 features as input.

In [16]:
# check predictions 
y_pred[:5]

NameError: name 'y_pred' is not defined

In [17]:
# get auc from the test set
auc_score = roc_auc_score(y_test, y_pred_prob, average='macro')

ValueError: multi_class must be in ('ovo', 'ovr')